In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from linear_geodesic_optimization.mesh.rectangle import Mesh as RectangleMesh
from linear_geodesic_optimization.optimization.curvature import Computer as Curvature
from linear_geodesic_optimization.optimization.geodesic import Computer as Geodesic
from linear_geodesic_optimization.optimization.laplacian import Computer as Laplacian
from linear_geodesic_optimization.optimization.smooth_loss import Computer as SmoothLoss
from linear_geodesic_optimization.plot import get_mesh_plot, get_heat_map

In [ ]:
width = 30
height = 30
scale = 10.
mesh = RectangleMesh(width, height, scale)
x, y, _ = mesh.get_coordinates().T
z = sum(
    scale * np.exp(-((x - mu[0])**2 + (y - mu[1])**2) * 25 / scale**2) / 5
    for mu in [
        (-scale / 4, -scale / 4),
        (-scale / 4, scale / 4),
        (scale / 4, -scale / 4),
        (scale / 4, scale / 4),
    ]
)
mesh.set_parameters(z)
get_mesh_plot(mesh)
plt.show()

In [ ]:
laplacian = Laplacian(mesh)
curvature = Curvature(mesh, laplacian)
smooth_loss = SmoothLoss(mesh ,laplacian, curvature)
smooth_loss.forward()
kappa = curvature.kappa_G
print(np.amin(kappa), np.amax(kappa))
print(smooth_loss.loss)

In [ ]:
vertices = mesh.get_coordinates()
x = list(sorted(set(x)))
y = list(sorted(set(y)))
z = vertices[:,2]

curvature.forward()
kappa = np.array(curvature.kappa_G).reshape(width, height).T
kappa[0,:] = 0.
kappa[-1,:] = 0.
kappa[:,0] = 0.
kappa[:,-1] = 0.
np.clip(kappa, -3, 3, kappa)
get_heat_map(x, y, kappa, 'Curvature')
plt.show()

In [ ]:
target_points = [
    (-scale / 4, -scale / 4),
    (-scale / 4, 0.),
    (-scale / 4, scale / 4),
    (0., -scale / 4),
    (0., 0.),
    (0., scale / 4),
    (scale / 4, -scale / 4),
    (scale / 4, 0.),
    (scale / 4, scale / 4),
]
target_curvatures = [
    (coordinate, kappa[mesh.nearest_vertex(coordinate).index()])
    for coordinate in target_points
]
print(target_curvatures)